# Task 4: Time series analysis for your power system

In task 4 of the project you are expected to assess to what extent your power system could operate under more realistic load and RES production conditions, and if RES could lead to the decommisioning of conventional power plants.

---

To do so, you need to find some representative hourly time series or Solar, Wind and Load values over a year. This may be easier or more difficult, depending on the area you chose to model. Remember that this is a crude assessment, so we care more about the overall characteristics of generation and demand, so accuracy is not so important. You can thus use similar datasets or proxies. This is very common in practice, as data is not always available.

---

Below we provide an example on how you can find, load and process the required datasets.  We will use **net demand** (*or residual load*) as the main variable to assess how much conventional generation is still required once renewable generation is considered, and to what extent your system can handle large variations.

**Net demand** is calculated as:

D(t) = L(t) - PV(t) - W(t)

where:  
- L(t): total system load at hour \(t\)  
- PV(t): solar (photovoltaic) generation at hour \(t\)  
- W(t): wind generation at hour \(t\)  

Positive D(t) values indicate that conventional generation is needed to meet the remaining demand, while negative values correspond to hours of renewable surplus (potential curtailment or export).

---

### RES Production Data
A great source is **[Renewables Ninja](https://renewables.ninja)**.
This website provides freely accessible hourly time series for solar and wind generation across many countries. You can either:

- Download national-level datasets for different years, or  
- Simulate production for a **specific location**, obtaining PV and wind generation **per kW of installed capacity** for a given year.

This flexibility makes Renewables.ninja an excellent tool for generating custom RES production profiles for any region or scenario.

We have already created and downloaded the two files. Below, we load the respective PV and Wind datasets (hosted on google drive) for 2019 (latest available year) and obtain a dataframe with 8760 hourly values.

In [1]:
# @title Script to load and process RES data
# Install gdown if not already installed
!pip install -q gdown

import gdown
import pandas as pd
from IPython.display import Markdown, display

# --- File 1 (PV) ---
file_id_1 = "1xAi-8Xef7LKOnDW2tsMvRyvS4kuXda5H"
url_1 = f"https://drive.google.com/uc?id={file_id_1}"
output_1 = "file1.csv"

gdown.download(url_1, output_1, quiet=True)

# --- File 2 (Wind) ---
file_id_2 = "12-aCDITEI9j4CrT_74XzSj4_Bm6ywL9v"
url_2 = f"https://drive.google.com/uc?id={file_id_2}"
output_2 = "file2.csv"

gdown.download(url_2, output_2, quiet=True)

# --- Load data, skipping metadata lines ---
df_PV = pd.read_csv(output_1, comment='#')
df_wind = pd.read_csv(output_2, comment='#')

# --- Keep only UTC time and electricity columns ---
df_PV = df_PV[['time', 'electricity']]
df_wind = df_wind[['time', 'electricity']]

# --- Rename electricity columns ---
df_PV.rename(columns={'electricity': 'PV'}, inplace=True)
df_wind.rename(columns={'electricity': 'Wind'}, inplace=True)

# --- Merge both datasets using UTC time ---
df_RES = pd.merge(
    df_PV, df_wind,
    on='time',
    how='inner'
)

df_RES['time'] = pd.to_datetime(df_RES['time'])

# --- Display confirmation and preview ---
display(Markdown("✅ **Script executed successfully.**  \nThe DataFrame looks like:"))
display(df_RES.head(5))

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

ModuleNotFoundError: No module named 'gdown'

### Consumption Data
There are various sources where you can obtain hourly values for demand.
A great starting point is **[ENTOE's transparency platform](https://transparency.entsoe.eu)**.
This website provides freely accessible 15-min or 60-min load time series for Europe.
Note that you need to create an account to extract data and save them; the account is free and the process takes one minute.

Below, we load the timeseries for the Dutch 2024 demand, containing 8784x4 15-min values im MW. We average to hourly values so that we have the same resolution with the RES data. You may skip this step if you find hourly time series instead.

In [ ]:
# @title Script to load and process Consumption data
# --- Download the load file from Google Drive ---
file_id_load = "1y12HXV_FWjriS3JGmPC0LMdL9TD68gzy"
url_load = f"https://drive.google.com/uc?id={file_id_load}"
output_load = "Load.csv"
gdown.download(url_load, output_load, quiet=True)

# --- Load CSV into a DataFrame ---
# Keep only the 'Time' and 'Actual Load' columns
df_load = pd.read_csv(output_load)
df_load = df_load[['Time (UTC)', 'Actual Total Load [MW] - Netherlands (NL)']]

# --- Rename columns for simplicity ---
df_load.columns = ['time', 'Load']

# --- Extract only the start time of each 15-minute interval ---
df_load['time'] = df_load['time'].str.split(' - ').str[0]

# --- Convert 'time' column to datetime format ---
df_load['time'] = pd.to_datetime(df_load['time'], format='%d.%m.%Y %H:%M')

# --- Set 'time' as index for resampling ---
df_load.set_index('time', inplace=True)

# --- Resample data to hourly intervals and take the mean ---
df_load = df_load.resample('h').mean().reset_index()

# --- Reset index after filtering and transformations ---
df_load = df_load.reset_index(drop=True)

# --- Display confirmation and preview ---
display(Markdown("✅ **Script executed successfully.**  \nThe DataFrame looks like:"))
display(df_load.head(5))

✅ **Script executed successfully.**  
The DataFrame looks like:

,time,Load
0,2024-01-01 00:00:00,11242.25
1,2024-01-01 01:00:00,11019.25
2,2024-01-01 02:00:00,10776.00
3,2024-01-01 03:00:00,10627.00
4,2024-01-01 04:00:00,10678.25


### Aligning the two time series

Now you have one dataframe for RES production and one for consumption. You can simply combine the two dataframes into one. If both time series correspond to the same year, merging them is easy. In our case we will simply change the year for the RES data to 2024 and remove the extra day from the consumption time series. For the type of analysis we are conducting, this simplification has no impact.

In [ ]:
# @title Script to align and merge the two dataframes
# --- Remove 29th of February ---
df_load = df_load[~((df_load['time'].dt.month == 2) &
                    (df_load['time'].dt.da  y == 29))]

df_load = df_load.reset_index(drop=True)
# --- Change the year to 2024 ---
df_RES['time'] = df_RES['time'].apply(lambda x: x.replace(year=2024))
df_RES = df_RES.reset_index(drop=True)

# --- Merge ---
df_merged = pd.merge(df_RES, df_load, on='time', how='inner')

# --- Display confirmation and preview ---
display(Markdown("✅ **Script executed successfully.**  \nThe DataFrame looks like:"))
display(df_merged.head(5))

✅ **Script executed successfully.**  
The DataFrame looks like:

,time,PV,Wind,Load
0,2024-01-01 00:00:00,0.0,0.963,11242.25
1,2024-01-01 01:00:00,0.0,0.973,11019.25
2,2024-01-01 02:00:00,0.0,0.980,10776.00
3,2024-01-01 03:00:00,0.0,0.983,10627.00
4,2024-01-01 04:00:00,0.0,0.983,10678.25


# Analysis

You are now ready to do your analysis.  
Your dataframe is called **`df_merged`** and contains the combined renewable energy and consumption data.

It includes the following columns:
- **`time`** – timestamps in UTC format.  
- **`PV`** – photovoltaic (solar) electricity generation values per installed kW.  
- **`Wind`** – wind electricity generation values per installed kW.
- **`Load`** - total consumption values in MW from the country you chose

You can now proceed with your analysis. For example:
- Calculate the min/max load scenarios (remember to normalize your load to an average of 4000 MW for a fair comparison)
- Calculate the max net demand hourly change in both directions. Which direction is easier to handle in terms of power system operation?
- How would you assess how many MW of conventional generation would you need?
- You can define some metrics for evaluating your system's performance and see which RES is more beneficial, or what is the optimal mix.

Below we provide two exemplary scripts.
In the first you can change the installed capacity of Wind and PV and see the residual load. In the second you can fix the installed capacities and observe the residual load in an interactive plot.

In [ ]:
# @title Script plot the residual demand with adjustable RES capacities
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Sliders
pv_slider = widgets.IntSlider(value=3000, min=0, max=6000, step=100, description='PV (MW)')
wind_slider = widgets.IntSlider(value=3000, min=0, max=6000, step=100, description='Wind (MW)')

# Plotting function
def plot_residual_load(pv_capacity_val, wind_capacity_val):
    df_test = df_merged.copy()
    df_test['Load'] = df_test['Load']*4000/df_test['Load'].mean()
    df_test['Residual_Load'] = df_test['Load'] - (pv_capacity_val * df_test['PV']) - (wind_capacity_val * df_test['Wind'])

    plt.figure(figsize=(18, 6))
    plt.plot(df_test['time'], df_test['Residual_Load'], label='Residual Load')
    plt.title(f'Residual Load (PV={pv_capacity_val} MW, Wind={wind_capacity_val} MW)')
    plt.xlabel('Time')
    plt.ylabel('MW')
    plt.grid(True)
    plt.show()

# Create auto-updating output
out = widgets.interactive_output(plot_residual_load, {
    'pv_capacity_val': pv_slider,
    'wind_capacity_val': wind_slider
})

# Display the UI
display(pv_slider, wind_slider, out)


IntSlider(value=3000, description='PV (MW)', max=6000, step=100)

IntSlider(value=3000, description='Wind (MW)', max=6000, step=100)

Output()

In [ ]:
# @title Interactive plot of residual demand
import plotly.express as px

pv_capacity_val = 3000  # MW, you can change this value to test
wind_capacity_val = 3000  # MW, change as needed

df_test = df_merged.copy()
df_test['Residual_Load'] = df_test['Load']*4000/df_test['Load'].mean() - (pv_capacity_val * df_test['PV']) - (wind_capacity_val * df_test['Wind'])

fig = px.line(df_test, x='time', y='Residual_Load',
              labels={'Residual_Load':'MW', 'time':'Time'},
              title=f"Residual Load Time Series (PV={pv_capacity_val} MW, Wind={wind_capacity_val} MW)")
fig.show()